In [1]:
import json
from pathlib import Path
import yaml

import pandas as pd
from tqdm.notebook import tqdm

### Load Data

In [2]:
def load_run_jsonl(infile):
    for l in open(infile):
        j = json.loads(l)
        obj = {k: v for k, v in j.items() if k in ['dataset', 'team', 'software', 'run_id']}
        obj['eval_field'] = 'all'
        obj['eval_value'] = 'all'
        eval_keys = [k for k in j['evaluation'] if k.startswith('_eval-')]
        obj_all = obj.copy()
        obj_all.update({'resource_' + k: v for k, v in j['used_resources'].items()})
        obj_all.update({k: v for k, v in j['evaluation'].items() if k not in eval_keys})
        yield obj_all

        for k in eval_keys:
            obj['eval_field'] = k.replace('_eval-', '')
            for kv, v in j['evaluation'][k].items():
                obj['eval_value'] = kv
                obj.update(**v)
                yield obj.copy()

df_run_eval = pd.DataFrame(load_run_jsonl('../data/tira/runs-gen-ai-authorship-verification/runs.jsonl')).set_index(['dataset', 'team', 'software', 'run_id'])
_resource_cols = [c for c in df_run_eval.columns if c.startswith('resource_')]
df_resources = df_run_eval[_resource_cols].dropna()
df_run_eval = df_run_eval.drop(columns=_resource_cols)
df_run_eval

eval_field  \
dataset                                            team        software           run_id                           
pan25-generative-ai-detection-eloquent-20250605... hello-world tart-objective     2025-06-05-12-11-02        all   
                                                                                  2025-06-05-12-11-02     source   
                                                                                  2025-06-05-12-11-02      model   
                                                                                  2025-06-05-12-11-02      model   
                                                                                  2025-06-05-12-11-02      model   
...                                                                                                          ...   
pan25-generative-ai-detection-20250604-test        ds-gt-pan   metallic-artillery 2025-06-04-17-23-58      genre   
                                                                                  2025-06-04-17-23-58      genre   
                                                                                  2025-06-04-17-23-58      genre   
                                                                                  2025-06-04-17-23-58      genre   
                                                                                  2025-06-04-17-23-58      genre   

                                                                                                          eval_value  \
dataset                                            team        software           run_id                               
pan25-generative-ai-detection-eloquent-20250605... hello-world tart-objective     2025-06-05-12-11-02            all   
                                                                                  2025-06-05-12-11-02       eloquent   
                                                                                  2025-06-05-12-11-02      Homoglyph   
                                                                                  2025-06-05-12-11-02  HumanAIzers-0   
                                                                                  2025-06-05-12-11-02  HumanAIzers-1   
...                                                                                                              ...   
pan25-generative-ai-detection-20250604-test        ds-gt-pan   metallic-artillery 2025-06-04-17-23-58         essays   
                                                                                  2025-06-04-17-23-58    essays-obfs   
                                                                                  2025-06-04-17-23-58        fiction   
                                                                                  2025-06-04-17-23-58   fiction-obfs   
                                                                                  2025-06-04-17-23-58           news   

                                                                                                       roc-auc  \
dataset                                            team        software           run_id                         
pan25-generative-ai-detection-eloquent-20250605... hello-world tart-objective     2025-06-05-12-11-02    0.937   
                                                                                  2025-06-05-12-11-02    0.937   
                                                                                  2025-06-05-12-11-02      NaN   
                                                                                  2025-06-05-12-11-02      NaN   
                                                                                  2025-06-05-12-11-02      NaN   
...                                                                                                        ...   
pan25-generative-ai-detection-20250604-test        ds-gt-pan   metallic-artillery 2025-06-04-17-23-58    0.973   
      

In [3]:
# Load case ID map
id_map = {}
for f in Path('../data/sampled').glob('*-orig-ids.jsonl'):
    for l in open(f, 'r'):
        j = json.loads(l)
        assert j['id'] not in id_map
        id_map[j['id']] = j['orig_id']

In [4]:
# Load raw raw predictions and back-translate case IDs
df_pred_raw = []
for pt in tqdm(list(Path('../data/tira/runs-gen-ai-authorship-verification').glob('*/run.prototext')), desc='Loading raw predictions'):
    y = yaml.full_load(open(pt))
    df_pred_raw.append(pd.read_json(next(pt.parent.glob('output/*.jsonl')), lines=True)[['id', 'label']])
    df_pred_raw[-1]['dataset'] = y['inputDataset']
    df_pred_raw[-1]['run_id'] = y['runId']
    r = df_run_eval.query('dataset == @y["inputDataset"] and run_id == @y["runId"]').index[0]
    df_pred_raw[-1]['team'] = r[1]
    df_pred_raw[-1]['software'] = r[2]
    df_pred_raw[-1]['id'] = df_pred_raw[-1]['id'].map(lambda x: id_map[x])

df_pred_raw = pd.concat(df_pred_raw).set_index(['dataset', 'team', 'software', 'run_id', 'id'])
df_pred_raw

Loading raw predictions:   0%|          | 0/148 [00:00<?, ?it/s]

label
dataset                                            team     software     run_id              id                                                          
pan25-generative-ai-detection-20250604-test        mdok     mdok         2025-06-04-16-11-23 gutenberg-19c-fiction/PG5673_9                      0.000308
                                                                                             riddell-juola-obfuscated/gpt-4o/59db82              1.000000
                                                                                             gutenberg-19c-fiction/deepseek-r1-distill-qwen-...  1.000000
                                                                                             brennan-greenstadt/gpt-4o-mini/j_12_4               1.000000
                                                                                             pan24-test/text-bison-002/news-2021-01-01-2021-...  1.000000
...                                                                                                                                                   ...
pan25-generative-ai-detection-eloquent-20250610... iimasnlp isg-graph-v3 2025-06-11-12-48-56 eloquent25/adhd/037                                 1.000000
                                                                                             eloquent25/adhd/038                                 0.901600
                                                                                             eloquent25/adhd/030                                 1.000000
                                                                                             eloquent25/adhd/033                                 0.991400
                                                                                             eloquent25/adhd/051                                 0.999900

[269130 rows x 1 columns]

### ELOQUENT Evaluation

In [5]:
eloquent_ds_names = [d for d in df_run_eval.reset_index()['dataset'].unique() if '-eloquent-' in d]
df_eval_eloquent = df_run_eval.query('dataset in @eloquent_ds_names')
df_eval_eloquent = df_eval_eloquent[df_eval_eloquent['eval_field'] == 'model']
df_eval_eloquent.to_csv('../data/tira/eloquent-stats.csv')
df_eval_eloquent

eval_field  \
dataset                                            team                software            run_id                           
pan25-generative-ai-detection-eloquent-20250605... hello-world         tart-objective      2025-06-05-12-11-02      model   
                                                                                           2025-06-05-12-11-02      model   
                                                                                           2025-06-05-12-11-02      model   
                                                                                           2025-06-05-12-11-02      model   
                                                                                           2025-06-05-12-11-02      model   
...                                                                                                                   ...   
pan25-generative-ai-detection-eloquent-20250610... cnlp-nits-pp        tomato-conduction   2025-06-11-00-11-42      model   
                                                   nexus-interrogators sensitive-liason    2025-06-11-00-35-49      model   
                                                   bohan-li            distinct-dachshund  2025-06-11-00-10-49      model   
                                                   ds-gt-pan           flavorful-concourse 2025-06-11-00-28-29      model   
                                                                       metallic-artillery  2025-06-11-00-31-16      model   

                                                                                                                   eval_value  \
dataset                                            team                software            run_id                               
pan25-generative-ai-detection-eloquent-20250605... hello-world         tart-objective      2025-06-05-12-11-02      Homoglyph   
                                                                                           2025-06-05-12-11-02  HumanAIzers-0   
                                                                                           2025-06-05-12-11-02  HumanAIzers-1   
                                                                                           2025-06-05-12-11-02  HumanAIzers-2   
                                                                                           2025-06-05-12-11-02  HumanAIzers-3   
...                                                                                                                       ...   
pan25-generative-ai-detection-eloquent-20250610... cnlp-nits-pp        tomato-conduction   2025-06-11-00-11-42           adhd   
                                                   nexus-interrogators sensitive-liason    2025-06-11-00-35-49           adhd   
                                                   bohan-li            distinct-dachshund  2025-06-11-00-10-49           adhd   
                                                   ds-gt-pan           flavorful-concourse 2025-06-11-00-28-29           adhd   
                                                                       metallic-artillery  2025-06-11-00-31-16           adhd   

                                                                                                                roc-auc  \
dataset                                            team                software            run_id                         
pan25-generative-ai-detection-eloquent-20250605... hello-world         tart-objective      2025-06-05-12-11-02      NaN   
                                                                                           2025-06-05-12-11-02      NaN   
                                                                                           2025-06-05-12-11-02      NaN   
                                                                                           2025-06-05-12-11-02      NaN   
                                                                     

In [6]:
df_eval_eloquent_mean = df_eval_eloquent[df_eval_eloquent['eval_field'] == 'model'].groupby('eval_value').mean(numeric_only=True).sort_values('mean', ascending=False)
df_eval_eloquent_mean.to_csv('../data/tira/eloquent-stats-mean.csv')
df_eval_eloquent_mean

,roc-auc,brier,c@1,f1,f05u,mean
eval_value,,,,,,
HumanAIzers-2,NaN,0.84512,0.81436,0.85888,0.90400,0.68452
Moa,NaN,0.78138,0.74960,0.81520,0.89176,0.64756
TeamUTK,NaN,0.78814,0.74118,0.80166,0.87424,0.64104
TeamUTK_bis,NaN,0.77414,0.73360,0.80422,0.88384,0.63916
HumanAIzers-3,NaN,0.78072,0.73082,0.79156,0.86042,0.63280
PJs-team-v1,NaN,0.72224,0.66560,0.75438,0.85100,0.59864
adhd,NaN,0.68386,0.64232,0.72598,0.82280,0.57488
HumanAIzers-1,NaN,0.59754,0.54884,0.59466,0.65450,0.47912
Homoglyph,NaN,0.59754,0.54884,0.59466,0.65450,0.47912


In [11]:
df_pred_raw_eloquent = df_pred_raw.query('dataset in @eloquent_ds_names')
df_pred_raw_eloquent.to_csv('../data/tira/eloquent-pred-raw.csv')
df_pred_raw_eloquent

label
dataset                                            team      software     run_id              id                                  
pan25-generative-ai-detection-eloquent-20250605... advacheck watery-bag   2025-06-05-11-20-09 eloquent25/humanaizers-0/047  0.0000
                                                                                              eloquent25/junlp_ss/033       1.0000
                                                                                              eloquent25/pjs-team-v2/031    0.0000
                                                                                              eloquent25/humanaizers-2/031  1.0000
                                                                                              eloquent25/moa/036            1.0000
...                                                                                                                            ...
pan25-generative-ai-detection-eloquent-20250610... iimasnlp  isg-graph-v3 2025-06-11-12-48-56 eloquent25/adhd/037           1.0000
                                                                                              eloquent25/adhd/038           0.9016
                                                                                              eloquent25/adhd/030           1.0000
                                                                                              eloquent25/adhd/033           0.9914
                                                                                              eloquent25/adhd/051           0.9999

[15065 rows x 1 columns]